In [ ]:
%load_ext autoreload
%autoreload 2

import pandas as pd
from IPython.core.display import HTML
from IPython.display import display
import torch
from tqdm import tqdm
from pathlib import Path

from oml.datasets.base import BaseDataset
from oml.models.vit.vit import ViTExtractor
from oml.transforms.images.albumentations.transforms import get_normalisation_resize_albu

from torch.utils.data import TensorDataset

display(HTML("<style>.container { width:100% !important; }</style>"))
pd.set_option('display.max_rows', 330)

%matplotlib inline

In [ ]:
dataset_root = Path("/nydl/data/DeepFashion_InShop/")
batch_size = 1024
weights = "vits16_dino"

df = pd.read_csv(dataset_root / "df_no_bboxes.csv")


In [ ]:
# dataset = BaseDataset(df=df, transform=get_normalisation_resize_albu(im_size=224))
# model = ViTExtractor(weights, arch="vits16", normalise_features=False).eval().cuda()
# train_loader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, shuffle=True)

# embeddings = torch.zeros((len(df), model.feat_dim))

# with torch.no_grad():
#     for i, batch in enumerate(tqdm(train_loader)):
#         embs = model(batch["input_tensors"].cuda()).detach().cpu()
#         ia = i * batch_size
#         ib = min(len(embeddings), (i + 1) * batch_size)
#         embeddings[ia:ib, :] = embs

        
# torch.save(embeddings, dataset_root / f"embeddings_{weights}.pkl")
        

In [ ]:
embeddings = torch.load(dataset_root / f"embeddings_{weights}.pkl")
dataset = TensorDataset(embeddings, torch.tensor(df["label"].to_numpy()))
